In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm


import matplotlib.pyplot as plt
import folium

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [49]:
def route_path(routeNo, dir):
    data_path = "/Users/nguyenduykhang/Documents/Projects/HCMC-Bus-Network/data/buyttphcm"
    if dir == 0:
        path = f"{data_path}/{routeNo}/paths_by_var.csv"
    else:
        path = f"{data_path}/{routeNo}/rev_paths_by_var.csv"
    
    path_df = pd.read_csv(path)
    path_df =path_df.join(path_df.shift(-1), rsuffix="_next").dropna()
    return(path_df)

def hub_station(routeNo, dir):
    data_path = "/Users/nguyenduykhang/Documents/Projects/HPCC-BusAlgos/data/bus-lines"
    hub_df = pd.read_csv(f"{data_path}/{routeNo}/{dir}_hub_nodes.csv")
    hub_df.drop(columns=['StopType', 'Zone', 'Ward', 'AddressNo', 'Street', 'SupportDisability', 'Status', 'Search', 'Routes'], axis=1, inplace=True)
    hub_df = hub_df.join(hub_df.shift(-1), rsuffix="_next").dropna()
    hub_df['StopId_next'] = hub_df['StopId_next'].astype(int)
    return hub_df

In [51]:
routeNo = 50
dir = 1

bus_path = route_path(routeNo=routeNo, dir=dir)
bus_hub = hub_station(routeNo=routeNo, dir=dir)

print(f"Bus RouteNo: {routeNo}")
print(bus_hub.info())
print(bus_path.info())

# get path and all bus stations --> visuz

Bus RouteNo: 50
<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StopId       28 non-null     int64  
 1   Code         28 non-null     object 
 2   Name         28 non-null     object 
 3   Lng          28 non-null     float64
 4   Lat          28 non-null     float64
 5   StopId_next  28 non-null     int64  
 6   Code_next    28 non-null     object 
 7   Name_next    28 non-null     object 
 8   Lng_next     28 non-null     float64
 9   Lat_next     28 non-null     float64
dtypes: float64(4), int64(2), object(4)
memory usage: 2.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 541 entries, 0 to 540
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   lat       541 non-null    float64
 1   lng       541 non-null    float64
 2   lat_next  541 non-null    float64
 3   lng_next  541 no

In [52]:
avg_location = bus_hub[['Lat', 'Lng']].mean()

hcmc_map = folium.Map(location=avg_location, zoom_start=12)

In [53]:
num_hub = len(bus_hub)
travel_time = 0

for point in bus_path.itertuples():
    marker = folium.CircleMarker(location=(point.lat, point.lng),
                                 radius=5,
                                 color='green',
                                 fill_color='green',
                                 fill=True,
                                 fill_opacity=1)
    line = folium.PolyLine(locations=[(point.lat, point.lng), (point.lat_next, point.lng_next)])
    
    marker.add_to(hcmc_map)
    line.add_to(hcmc_map)

for hub in bus_hub.itertuples():
    OD_hub = (hub.Index == 0) or (hub.Index == num_hub)
    # maker for the current hub
    icon = folium.Icon(icon='home' if OD_hub else 'info-sign',
                       color='cadetblue' if OD_hub else 'green')
    
    marker = folium.Marker(location=(hub.Lat, hub.Lng),
                          icon=icon,
                          tooltip=f"<b>StopID</b>: {hub.StopId} <br>" \
                              + f"<b>Name</b>: {hub.Name} <br>")
    line = folium.PolyLine(locations=[(hub.Lat, hub.Lng), (hub.Lat_next, hub.Lng_next)],
                           color='red',
                           tooltip=f"<b>From</b>: {hub.StopId} <br>" \
                                + f"<b>To</b>: {hub.StopId_next} <br>" \
                                + f"<b>Travel Time</b>: {travel_time} s")
    
    marker.add_to(hcmc_map)
    line.add_to(hcmc_map)

folium.Marker(location=(hub.Lat_next, hub.Lng_next),
              icon=folium.Icon(icon='home', color='cadetblue'),
              tooltip=f"<b>StopID</b>: {hub.StopId_next} <br>" \
                  + f"<b>Name</b>: {hub.Name_next} <br>").add_to(hcmc_map)

hcmc_map

In [ ]:
# function to get and visuz map
